In [2]:
import numpy as np
import open3d as o3d
import os
import pandas as pd
from tqdm import tqdm
import pickle
from collections import defaultdict

o3d.utility.set_verbosity_level(o3d.cpu.pybind.utility.VerbosityLevel(0))

In [3]:
dataset = pd.read_pickle('../data/4877.pickle')

In [23]:
def pcd_to_voxel(xyz, dim_size=128, full_volume=False, plot=False):

    if full_volume:
        # scale in every dimension separatly to fit in the volume
        for d in range(3):
            dmin = xyz[:,d].min()
            dmax = xyz[:,d].max()
            xyz[:,d] = (xyz[:,d] - dmin) / (dmax - dmin) * dim_size
    else:
        # scale every dimension by the same factor to preserve aspect ratio
        ratio = float('inf')

        for d in range(3):
            dmin = xyz[:,d].min()
            dmax = xyz[:,d].max()
            cand_ratio = dim_size/(dmax - dmin)
            if ratio > cand_ratio:
                ratio = cand_ratio
            xyz[:,d] = (xyz[:,d] - dmin)  

        xyz *= ratio
        # and center the object in just one plane
        for d in [0,2]:
            dmin = xyz[:,d].min()
            dmax = xyz[:,d].max()
            xyz[:,d] += (dim_size - (dmax - dmin))/2

    xyz = np.rint(xyz)
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(xyz)

    if plot:
        bbox = o3d.geometry.AxisAlignedBoundingBox(np.array([0,0,0]), np.array([dim_size,dim_size,dim_size]))
        bbox.color = (255,0,0)
        o3d.visualization.draw_geometries([pcd, bbox])

    return pcd

In [26]:
normalized_pcds = []

# iterate over dataframe and convert each pointcloud to voxel, add to new column in dataframe
for i in tqdm(range(len(dataset))):
    xyz = dataset.loc[i]['Points_coordinates'].copy()
    normalized_pcds.append(pcd_to_voxel(xyz, dim_size=128, full_volume=False, plot=False))

dataset['voxel'] = normalized_pcds

100%|██████████| 4877/4877 [01:01<00:00, 78.98it/s] 
